# Create and Run chat flows using Promptflow Python SDK 
프롬프트 플로우는 아이디어 발상, 프로토타이핑, 테스트, 평가부터 프로덕션 배포 및 모니터링에 이르기까지 LLM 기반 AI 애플리케이션의 엔드투엔드 개발 주기를 간소화하도록 설계된 개발 도구 모음입니다. 이를 통해 신속한 엔지니어링이 훨씬 쉬워지고 프로덕션 수준의 품질로 LLM 앱을 빌드할 수 있습니다. 

> ✨ ***Note*** <br>
> Promptflow는 오픈소스 프로젝트이며, Azure ML, Azure AI Studio, Local VS Code환경을 모두 지원합니다.  

## Prerequisites
3.10 또는 이후 버전의 Python 가상 환경을 구성합니다 : 
 1. 명령 Palette를 엽니다 (Ctrl+Shift+P).
 1. Python을 검색합니다: Create Environment를 선택합니다.
 1. Venv / Conda를 선택하고 새 환경을 만들 위치를 선택합니다.
 1. Python interpreter 버전을 선택합니다. 3.10 이상의 버전으로 생성합니다.

디펜던시 설치를 위해 아래 코드를 실행하여 실행에 필요한 package들을 설치하세요. 

```bash
pip install -r requirements.txt
```

## Set up your environment
현재의 repository를 Clone 하세요. 

```bash
git clone https://github.com/hyogrin/Azure_OpenAI_samples.git
```

.env-sample 파일을 기반으로 .env 파일을 만듭니다. 새 .env 파일을 노트북이 들어 있는 폴더에 복사하고 변수를 업데이트합니다.

## Steps
#### 1. Set up Promptflow client with Credential and configuration
#### 2. Create a new chat flow by providing the flow name and description.
#### 3. Run Basic Promptflow with questions to compare models
#### 4. Run Context Added Promptflow with the outdoor questions 
#### 5. Use serverless endpoint to run the Promptflow with context

In [ ]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

In [ ]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


## 2. Create a new chat flow by providing the flow name and description.

-   Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI.

> ✨ **_important_** <br>
> Grant the Storage File Data Privileged Contributor, Storage Blob Data Contributor at the storage of AI studiorole to user, group, service principle and managed Identity which you are trying to access, you can execute the code below.


### Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
> ✨ ***important*** <br>
> Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.

In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_aoai_connection_name": "replace with your connection name",
}

rendered_content = template.render(variables)
Path('./chat/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat/flow.dag.yaml').read_text()) 

In [ ]:
#Grant the right accessibility to create the flow 
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

## 3. Run Basic Promptflow with questions to compare models

-   Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models

In [ ]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result)

In [ ]:
detail = pf_azure_client.get_details(run_result)

detail

## 4. Run Context Added Promptflow with the outdoor questions

-   Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-context.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_aoai_connection_name": "replace with your connection name",
}

rendered_content = template.render(variables)
Path('./chat-context/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-context/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result_with_context)

In [ ]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

## 4. Use serverless endpoint to run the Promptflow with context
- Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.

### deploy your serverless endpoint

- go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint 
<br>
![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
<br>
<br>
- once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
![copy connection](./images/copy_connection.jpg)
<br>
<br>
- go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
![create new serverless connection](./images/create_new_serverless_connection.jpg)





### Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
> ✨ ***important*** <br>
> Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.

In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-serverless.flow.dag.yaml')

# Define the variables for the template with your connection names for chat serverless 
variables = {
	"your_phi35_serverless_connection_name": "replace with your connection name",
	"your_aoai_connection_name": "replace with your connection name",
}

rendered_content = template.render(variables)
Path('./chat-serverless/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-serverless/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_serverless_result)

In [ ]:
detail = pf_azure_client.get_details(run_serverless_result)

detail